# Explore Marine Data

Want to explore [NCEI Data](https://www.ncei.noaa.gov/cdo-web/datasets), specifically the [marine](https://www.ncei.noaa.gov/data/global-marine/) dataset. What is in this dataset

In [1]:
import duckdb
import glob

## Download

Use the scripts in the data dir to download the data. Run `aria2c -i urls.txt` to download the data. Decompress with the `.sh` script. You will need to make dir's.

## Load Data

In [2]:
marine_dir = "/home/squirt/Documents/data/ncei/marine/marine_data/"

Load into duckdb

Data columns in sample are not the same in the real data. WTF. Im also not sure if these CSV columns are the same accross all csvs?? Lol. The real world is messy.

In [3]:
all_csv_columns = ["STATION", "DATE", "LATITUDE", "LONGITUDE", "ELEVATION", "NAME", "IMMA_VER", "ATTM_CT", "TIME_IND", "LL_IND", "SHIP_COURSE", "SHIP_SPD", "ID_IND", "COUNTRY_CODE", "WIND_DIR_IND", "WIND_DIR", "WIND_SPD_IND", "WIND_SPEED", "VV_IND", "VISIBILITY", "PRES_WX", "PAST_WX", "SEA_LVL_PRES", "CHAR_PPP", "AMT_PRES_TEND", "IND_FOR_TEMP", "AIR_TEMP", "IND_FOR_WBT", "WET_BULB_TEMP", "DPT_IND", "DEW_PT_TEMP", "SST_MM", "SEA_SURF_TEMP", "TOT_CLD_AMT", "LOW_CLD_AMT", "LOW_CLD_TYPE", "HGT_IND", "CLD_HGT", "MID_CLD_TYPE", "HI_CLD_TYPE", "WAVE_PERIOD", "WAVE_HGT", "SWELL_DIR", "SWELL_PERIOD", "SWELL_HGT", "TEN_BOX_NUM", "ONE_BOX_NUM", "DECK", "SOURCE_ID", "PLATFORM_ID", "DUP_STATUS", "DUP_CHK", "NIGHT_DAY_FLAG", "TRIM_FLAG", "NCDC_QC_FLAGS", "SOURCE_EXCLUSION_FLAG", "OB_SOURCE", "OB_PLATFORM", "FM_CODE_VER", "STA_WX_IND", "PAST_WX2", "DIR_OF_SWELL2", "PER_OF_SWELL2", "HGT_OF_SWELL2", "IND_FOR_PRECIP", "QC_IND", "QC_IND_FOR_FIELDS", "MQCS_VER"]

Make DuckDB table to store data 

In [4]:
conn = duckdb.connect(database=":memory:", read_only=False)
table_name = 'marine_climate_data'

# Create DuckDB table
table_columns = "Station VARCHAR, Time DATETIME, Lat DOUBLE, Lon DOUBLE, WindSpeed DOUBLE, AirTemp DOUBLE, WetTemp DOUBLE, SeaTemp DOUBLE, CloudAmount DOUBLE"
conn.execute(f"CREATE TABLE {table_name} ({table_columns})")

Read in CSV Columns to DuckDB

In [5]:
# Get CSV files
csv_files = glob.glob(marine_dir + "*.csv")

# Need to map the CSV columns to the DuckDB table columns
csv_columns = ["STATION", "DATE", "LATITUDE", "LONGITUDE", "WIND_SPEED", "AIR_TEMP", "WET_BULB_TEMP", "SEA_SURF_TEMP", "TOT_CLD_AMT"] 
temp_table = 'temp_table'

for csv_file in csv_files:
    # Create a temporary table from the CSV file
    conn.execute(f"CREATE TABLE {temp_table} AS SELECT * FROM read_csv_auto('{csv_file}')")

    # Drop table if columns not present (because I don't really understand this data)
    # Fetch the column names from the table
    table_info = conn.execute(f"PRAGMA table_info({temp_table})").fetchall()
    temp_table_columns = [column[1] for column in table_info]

    # Compare the table's columns with the csv_columns list
    if set(csv_columns) - set(temp_table_columns):
        # Drop the table if the columns don't match
        conn.execute(f"DROP TABLE IF EXISTS {temp_table}")
    else:
        # Insert data from temporary table into final table with column mapping and type conversion
        query = f"""
        INSERT INTO {table_name} (Station, Time, Lat, Lon, WindSpeed, AirTemp, WetTemp, SeaTemp, CloudAmount)
        SELECT 
            STATION, 
            TRY_CAST(REPLACE(DATE, 'T', ' ') AS DATETIME), 
            CAST(LATITUDE AS DOUBLE), 
            CAST(LONGITUDE AS DOUBLE), 
            CAST(WIND_SPEED AS DOUBLE), 
            CAST(AIR_TEMP AS DOUBLE), 
            CAST(WET_BULB_TEMP AS DOUBLE), 
            CAST(SEA_SURF_TEMP AS DOUBLE), 
            CAST(TOT_CLD_AMT AS DOUBLE)
        FROM {temp_table}
        """
        conn.execute(query)
    
        # Drop the temporary table after all CSV files have been imported
        conn.execute(f"DROP TABLE {temp_table}")

To verify data was ingested print out the count

In [6]:
print(conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchall())

[(511470,)]
